# Project
Ziqi Gao/ Yibin Zhao/ Siyu Pan

This project asks us to construct an algorithm to track the SPY index on Quantopian platform. 
The first thought is that we can construct a portfolio using the top 20 stocks with highest market captalization since the SPY tracks the S&P 500 stock market index. We set our base universe to Q500US.
We use data from Jan 2005 to Dec 2015 to test our algorithm.


We record the tracking error as the difference between the portfolio return and the SPY return. 
We calculated the SPY return as (SPY price-SPY initial price)/SPY initial price

We set the universe updated frenquency to monthly. At first day of each month, we updated our universe by buying top 20 stocks with highest market captalization. Within the month, if some of the stocks becomes unavailable, we reduce our secuity list. When the new month begin, we renew our universe with buying 20 stocks with highest market captalization. Instead of buying equal weight, we buy weight respected to market captalization. We buy more on weight with the stock with higher market captalization. Define:
weight = market cap/Sum of market cap of tradable stock in our portfolio
We set our trading frequency to daily. We rebalance our portfolio to the weight defined above everyday.

From the testing result we find out that the tracking error is getting bigger as time elapse. We did some research and find out that the benchmark represent the total return and reivests dividends. However, the return we calculated does not contain the dividends reivested so there will be more tracking error as the dividend cumulated. 

Using the algorithm on the testing period which is Jan 2016 to Dec 2017, we obtain a portfolio with beta 0.94 which is close to 1. This means that our portfolio performs similar to the market. Moreover, we have a tracking error around 2% which is relatively small. 

In [ ]:
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.filters import Q500US
from quantopian.pipeline import CustomFactor
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import Returns

class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = close[-1] * shares[-1]

def initialize(context):
    schedule_function(
        rebalance, 
        date_rules.every_day(),
        time_rules.market_open(hours = 1))    
    schedule_function(
        record_vars,
        date_rules.every_day(),
        time_rules.market_close())
    schedule_function(
        renew_universe,
        date_rules.month_start(),  
        time_rules.market_open())    
    # Create our dynamic stock selector.
    set_benchmark(symbol('SPY'))
    pipe=make_pipeline()
    attach_pipeline(pipe, 'pipeline')
    context.security_list=[]
    context.update_signal = True
    
def make_pipeline():
    universe = Q500US()
    mkt_cap = MarketCap()
    top_mc_stocks = mkt_cap.top(20, mask=universe)
    securities_to_trade = (top_mc_stocks)
    pipe = Pipeline(
              columns={
                'market_capitalization': mkt_cap,
                'longs': top_mc_stocks,
              },
              screen = securities_to_trade
          )
    return pipe

def renew_universe(context,data):
    #function used to rebalanced at the begining of every month
    now = get_datetime()
    log.info("now.month.......... ---> {0}",now.month)
    weight = compute_weights(context,data)
    count=0
    for stock in context.security_list:
        if data.can_trade(stock):
            if stock in context.longs.index:
                order_target_percent(stock, weight[count])
                count+=1
    for stock in context.old_security_list:
        if stock not in context.security_list and data.can_trade(stock):
            log.info("need remove....{0}",stock)
            order_target_percent(stock, 0)

def compute_weights(context,data):
    sum=0
    w=[]
    for stock in context.security_list:
        if data.can_trade(stock):
            for mkt_cap in context.mktcap:
                sum+=mkt_cap
        if data.can_trade(stock):
            for mkt_cap in context.mktcap:
                weights = mkt_cap/sum
                w.append(weights)          
    return w

def before_trading_start(context, data):
    context.output = pipeline_output('pipeline')    
    context.longs = context.output[context.output['longs']]
    context.mktcap=context.output['market_capitalization']
    context.old_security_list=context.security_list
    context.security_list = context.longs.index.tolist()             
    for stock in context.security_list:
        if data.can_trade(stock) == False:
            context.stocks.remove(stock)

def rebalance(context, data):
    weight = compute_weights(context,data) 
    count=0
    for stock in context.security_list:
        if data.can_trade(stock):
            if stock in context.longs.index:
                order_target_percent(stock, weight[count])
                count+=1
    for stock in context.old_security_list:    
        if stock not in context.security_list and data.can_trade(stock):
            order_target_percent(stock, 0)

def record_vars(context, data):
    initial_spy=context.initial_spy
    SPY=(data.current(sid(8554), 'price')-initial_spy)/initial_spy
    record(tracking_error=(context.portfolio.returns-SPY)*100
          )

def handle_data(context, data):
    if context.update_signal:
        context.initial_spy=data.current(sid(8554), 'price')
        context.update_signal=False